In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Ruta al archivo shapefile de las AGEBs de la CDMX
agebs_shapefile_path = "C:/Users/MI18877/Auto-GPT/auto_gpt_workspace/Tesis/datos/shapefiles/ancilatorios/Vegetacion/usoveg_2020.shp"

# Leer el archivo shapefile de las AGEBs
vegetacion = gpd.read_file(agebs_shapefile_path)

vegetacion['VegUSo'].nunique()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

# Leer el archivo shapefile de las AGEBs
vegetacion = gpd.read_file(agebs_shapefile_path)

# Crear un mapa de colores para los valores únicos en VegUSo usando el mapa de colores 'tab20'
unique_veguso_values = vegetacion["VegUSo"].unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_veguso_values)))
color_map = dict(zip(unique_veguso_values, colors))

# Asignar el color gris a todos los valores de VegUSo que contengan la palabra "Urbano"
for veguso in unique_veguso_values:
    if "Urbano" in veguso:
        color_map[veguso] = (0.5, 0.5, 0.5, 1)

# Crear la figura y el eje para la visualización
fig, ax = plt.subplots(figsize=(120, 120))

# Visualizar el shapefile con colores según la columna "VegUSo_index" y el mapa de colores modificado
vegetacion["VegUSo_index"] = vegetacion["VegUSo"].apply(lambda x: np.where(unique_veguso_values == x)[0][0])
vegetacion.plot(column="VegUSo_index", ax=ax, categorical=True, cmap=plt.cm.tab20)

# Establecer el color de los polígonos en función del mapa de colores
for veguso, color in color_map.items():
    vegetacion[vegetacion["VegUSo"] == veguso].plot(ax=ax, color=color)

# Crear una lista de objetos Patch con los colores y etiquetas correspondientes a cada valor único de VegUSo
legend_patches = [Patch(facecolor=color, edgecolor='k', label=veguso) for veguso, color in color_map.items()]

# Agregar la leyenda personalizada
ax.legend(handles=legend_patches, loc='upper left', bbox_to_anchor=(1, 1), title='VegUSo')

# Agregar título y etiquetas
ax.set_title("Vegetación y uso de suelo en la Ciudad de México")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

# Mostrar la visualización
plt.show()

In [5]:
pesos_uso_suelo = {
    'Agricultura/Asentamiento': 2,
    'Agricultura/Bosque': 1,
    'Amaranto-Maiz/Avena-Ebo': 1,
    'Asentam/Agricultura': 2,
    'Avena': 1,
    'Avena-Papa/Maiz': 1,
    'Avena-pastos': 1,
    'B-Oyamel': 1,
    'Bosque cultivado': 1,
    'Bosque cultivado/Asentam': 2,
    'Bosque Encino': 1,
    'Bosque mixto': 1,
    'Bosque Pino': 1,
    'Bosque/Agricultura1': 1,
    'Bosque/Asentamiento': 2,
    'Canal-Lago': 1,
    'Cantera': 1,
    'Hortalizas-Maiz-Avena/borregos': 1,
    'Hortalizas/Flores': 1,
    'Humedal': 1,
    'Maiz/hortalizas': 1,
    'Maiz-frijol/pastos-ganado vacuno': 1,
    'Maiz-Maguey/avena/Haba-hortalizas': 1,
    'Maiz-Nopal/Asentamiento': 2,
    'Maiz-nopal/Matorral': 1,
    'Maiz/Avena-Alfalfa-Ebo': 1,
    'Maiz/frutales/ornamentales': 1,
    'Matorral': 1,
    'Matorral-Pastizales/Maiz': 1,
    'Nopal': 1,
    'Pastizal-Zacate': 1,
    'Pastizal/Urbano': 2,
    'Pinos/pastizales': 1,
    'Riego (romerito/brocoli/amaranto)': 1,
    'Riego/hortalizas- acelga-espinaca': 1,
    'VAah-Arboreo,arbustivo': 1,
    'VSA/Bosque pino': 1,
    'VSA/Oyamel': 1,
    'VSAa/Bosque Mixto': 1,
    'ZU': 3,
    'Avena/Maiz': 1,
    'Urbano': 3,
}


In [6]:
# Agregar una nueva columna 'peso' al GeoDataFrame y asignar los pesos correspondientes
vegetacion['pesos_uso_suelo'] = vegetacion['VegUSo'].map(pesos_uso_suelo)

# Guardar el GeoDataFrame con los pesos en un nuevo archivo .shp
vegetacion.to_file('vegetacion.shp')

C:\Users\MI18877\AppData\Local\Temp\1\ipykernel_28608\2233565177.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vegetacion.to_file('vegetacion.shp')


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio import features
from rasterio.plot import show


# Crear una nueva columna en el GeoDataFrame con el valor numérico correspondiente
vegetacion['intensity_value'] = vegetacion['pesos_uso_suelo'].astype(int)

# Rasterizar el archivo vectorial (shapefile)
def rasterize_geodataframe(geodataframe, attribute, resolution=30):
    bounds = geodataframe.total_bounds
    transform = rasterio.transform.from_origin(bounds[0], bounds[3], resolution, resolution)
    raster_shape = (int((bounds[3] - bounds[1]) / resolution),
                    int((bounds[2] - bounds[0]) / resolution))

    out_raster = np.zeros(raster_shape, dtype=np.uint8)

    shapes = ((geom, value) for geom, value in zip(geodataframe.geometry, geodataframe[attribute]))
    burned = features.rasterize(shapes=shapes, fill=0, out=out_raster, transform=transform, dtype=np.uint8)

    return burned, transform

raster, transform = rasterize_geodataframe(vegetacion, 'intensity_value', resolution=30)

output_raster_path = 'raster_VegUso.tif'

with rasterio.open(output_raster_path, 'w', driver='GTiff', width=raster.shape[1], height=raster.shape[0],
                  count=1, crs=vegetacion.crs, transform=transform, dtype=raster.dtype) as dst:
    dst.write(raster, indexes=1)


# Leer el archivo raster temporal
with rasterio.open('temp.tif') as src:
    img = src.read(1)

# Visualizar el archivo rasterizado
fig, ax = plt.subplots(figsize=(120, 120))

show(img, transform=src.transform, ax=ax, cmap=plt.cm.Blues)

ax.set_title("Peso de uso de suelo en relación con la distribución de la población en la Ciudad de México (rasterizado)")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

plt.show()

In [ ]:
# Leer el archivo raster creado previamente
with rasterio.open('C:/Users/MI18877/Auto-GPT/auto_gpt_workspace/Tesis/scripts/population_mex_2018-10-01.tif') as src:
    saved_raster_data = src.read(1)
    raster_transform = src.transform

# Visualizar el archivo rasterizado
fig, ax = plt.subplots(figsize=(12, 12))

show(saved_raster_data, transform=raster_transform, ax=ax, cmap=plt.cm.Blues)

ax.set_title("Peso de uso de suelo en relación con la distribución de la población en la Ciudad de México (rasterizado)")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

plt.show()
